In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from eipy.deep_ei import EnsembleIntegration

2023-11-09 20:47:48.166334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 20:47:48.195224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 20:47:48.195257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 20:47:48.195280: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 20:47:48.201258: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 20:47:48.201804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
y_train = np.where(y_train != 1, 0, y_train)
y_test = np.where(y_test != 1, 0, y_test)

In [5]:
import pickle as pkl
with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [6]:
'''from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}'''

"from sklearn.utils.class_weight import compute_class_weight\nclass_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)\nclass_weight_dict = {i: weight for i, weight in enumerate(class_weights)}"

In [7]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
res_model = ResNet50(weights='imagenet', include_top=False)

for layer in res_model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)

res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
'''from skimage.transform import resize

new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')
'''

"from skimage.transform import resize\n\nnew_size = (224, 224)\n\nX_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))\nX_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))\n\nfor i in range(X_train.shape[0]):\n    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')\nfor i in range(X_test.shape[0]):\n    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')\n"

In [9]:
'''with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:
    pkl.dump(X_train_resized, file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:
    pkl.dump(X_test_resized, file=file)
file.close()'''

'with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:\n    pkl.dump(X_train_resized, file=file)\nfile.close()\nwith open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:\n    pkl.dump(X_test_resized, file=file)\nfile.close()'

In [10]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
net_model = Model(inputs=base_model.input, outputs=predictions)

net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
base_predictors = {
    "ResNet50" : res_model,
    "MobileNet" : net_model
}

In [12]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=2,
                        n_samples=1,
                        sampling_aggregation=None,
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="images",
                        model_building=False,
                        )

In [13]:
EI.fit_base(X_train_resized, y_train)

Training base predictors on None...
        
... for ensemble performance analysis...


Generating ensemble training data: |          |  0%

INFO:tensorflow:Assets written to: ram://d158b73fb0344074934469a9467250b5/assets


INFO:tensorflow:Assets written to: ram://d158b73fb0344074934469a9467250b5/assets


INFO:tensorflow:Assets written to: ram://d80f0521a9524ac48dc788d9feaf0062/assets


...

INFO:tensorflow:Assets written to: ram://a77aafe0c8ee4554880336aa17bde4bf/assets


INFO:tensorflow:Assets written to: ram://a77aafe0c8ee4554880336aa17bde4bf/assets
2023-11-09 20:49:04.631795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 20:49:04.658782: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 20:49:04.658823: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 20:49:04.658846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 20:49:04.664517: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-0

ValueError: Found array with dim 4. None expected <= 2.

INFO:tensorflow:Assets written to: ram://37251d84502a41878996114b4779a30d/assets


INFO:tensorflow:Assets written to: ram://37251d84502a41878996114b4779a30d/assets
